In [123]:
from IPython.display import Markdown, display
import json

In [38]:
from langgraph_sdk import get_client

# Connect via SDK
url_for_cli_deployment = "http://localhost:8123"
client = get_client(url=url_for_cli_deployment)

In [136]:
def show_chunk_result(chunk):
    if chunk.data and chunk.event != "metadata" and len(chunk.data["parent_ids"]) == 1:
        event_type = chunk.data["event"]
        node_name = chunk.data["name"]
        data = chunk.data["data"]

        if event_type == "on_chain_start":
            print(f"Node '{node_name}' execution started.")
            # print(json.dumps(chunk, indent=2))
        elif event_type == "on_chain_end":
            print(f"Node '{node_name}' execution ended.")
            # print(json.dumps(chunk, indent=2))
        elif event_type == "on_chain_stream":
            print(f"Node '{node_name}' Stream.")
            # print(json.dumps(chunk, indent=2))
            analysts = data["chunk"].get('analysts', '')
            if analysts:
                for analyst in analysts:
                    print(f"Name: {analyst['name']}")
                    print(f"Affiliation: {analyst['affiliation']}")
                    print(f"Role: {analyst['role']}")
                    print(f"Description: {analyst['description']}")
                    print("-" * 50)
            final_report = data["chunk"].get('final_report', '')
            if final_report:
                display(Markdown(final_report))


Initial call to the graph

In [137]:
max_analysts = 3
topic = "The best comfy and simulation games of 2024 in steam"
config = {"configurable": {"user_id": "test2"}}
graph_name = "researcher"

thread = await client.threads.create()
print(f"Thread ID: {thread['thread_id']}")

chunks = []
async for chunk in client.runs.stream(thread["thread_id"], graph_name, input={"topic":topic, "max_analysts":max_analysts}, stream_mode="events", config=config):
    chunks.append(chunk)
    try:   
        show_chunk_result(chunk)
    except Exception as e:
        print(f"Error: {e}")
        print(chunk)
        break

Thread ID: ab6fcabd-c9e9-4c6d-a943-46cec9159eb0
Node 'create_analysts' execution started.
Node 'create_analysts' Stream.
Name: Alex Thompson
Affiliation: Game Review Hub
Role: Game Critic
Description: Alex focuses on evaluating the narrative and artistic elements of comfy and simulation games, emphasizing player experience and emotional engagement.
--------------------------------------------------
Name: Jordan Lee
Affiliation: Gamer Insights
Role: Market Analyst
Description: Jordan analyzes market trends and player demographics, providing insights into the popularity and sales potential of comfy and simulation games on Steam.
--------------------------------------------------
Name: Samira Patel
Affiliation: Tech & Play
Role: Technical Analyst
Description: Samira examines the technological aspects of game development, including graphics, mechanics, and user interface design, to assess how these elements contribute to the comfort and immersive experience of games.
----------------------

In [ ]:
# for chunk in chunks:
#     try:   
#         show_chunk_result(chunk)
#     except Exception as e:
#         print(f"Error: {e}")
#         print(chunk)
#         break


Give human feedback and run again

In [138]:
analysts_feedback = "Instead of the game critic, add an indie game player"

await client.threads.update_state(
  thread["thread_id"],
  {"human_analyst_feedback": analysts_feedback},
  as_node="human_feedback"
)

async for chunk in client.runs.stream(thread["thread_id"], graph_name, stream_mode="events"):
    try:   
        show_chunk_result(chunk)
    except Exception as e:
        print(f"Error: {e}")
        print(json.dumps(chunk, indent=2))
        break

Node 'create_analysts' execution started.
Node 'create_analysts' Stream.
Name: Liam Carter
Affiliation: Indie Game Community
Role: Indie Game Enthusiast
Description: Liam is passionate about discovering and promoting indie games that offer unique and cozy experiences. He focuses on gameplay mechanics, artistic styles, and community engagement, aiming to highlight hidden gems in the indie game scene.
--------------------------------------------------
Name: Sophie Chen
Affiliation: Gaming Trends Blog
Role: Simulation Game Specialist
Description: Sophie analyzes the latest simulation games, focusing on their immersive qualities and player engagement. She is particularly interested in how these games create relaxing environments and foster creativity among players.
--------------------------------------------------
Name: Ethan Rivera
Affiliation: Steam User Reviews
Role: Casual Gamer Advocate
Description: Ethan represents the voice of casual gamers on Steam, emphasizing the importance of c

Continue the graph execution when there is no more feedback

In [139]:
await client.threads.update_state(
  thread["thread_id"], 
  {"human_analyst_feedback": None},
  as_node="human_feedback"
)

async for chunk in client.runs.stream(thread["thread_id"], graph_name, stream_mode="events"):
    try:   
        show_chunk_result(chunk)
    except Exception as e:
        print(f"Error: {e}")
        print(chunk)
        break

Node 'conduct_interview' execution started.
Node 'conduct_interview' execution started.
Node 'conduct_interview' execution started.
Node 'conduct_interview' Stream.
Node 'conduct_interview' execution ended.
Node 'conduct_interview' Stream.
Node 'conduct_interview' execution ended.
Node 'conduct_interview' Stream.
Node 'conduct_interview' execution ended.
Node 'write_report' execution started.
Node 'write_introduction' execution started.
Node 'write_conclusion' execution started.
Node 'write_conclusion' Stream.
Node 'write_conclusion' execution ended.
Node 'write_introduction' Stream.
Node 'write_introduction' execution ended.
Node 'write_report' Stream.
Node 'write_report' execution ended.
Node 'finalize_report' execution started.
Node 'finalize_report' Stream.


# Embracing Comfort: The Best Comfy and Simulation Games of 2024 on Steam

## Introduction

As the gaming landscape evolves, 2024 has emerged as a remarkable year for cozy and simulation games on Steam. This report delves into the charm of cozy indie games, highlighting their emphasis on relaxation, creativity, and community engagement. We explore standout titles that prioritize comfort, such as city-building simulators and productivity games designed for a laid-back experience. Additionally, we examine the allure of immersive environments and customization options that define this genre. With a growing community of players seeking solace in these virtual spaces, cozy gaming is not just a trend—it's a movement that enriches the gaming experience.

---


The cozy gaming genre has seen a remarkable rise in popularity in 2024, particularly on platforms like Steam, where players are increasingly drawn to experiences that prioritize relaxation, creativity, and community engagement. This shift reflects a broader trend in the gaming landscape, as casual gamers seek refuge from the high-pressure environments often associated with mainstream titles.

Cozy indie games are characterized by their charming aesthetics and engaging gameplay mechanics, which allow players to unwind while participating in light-hearted activities. Notable titles emerging in 2024 include city-building simulators that enable players to create quaint towns filled with adorable characters, as well as productivity games designed for the "vibe crowd," featuring extensive customization options and soothing soundtracks [1]. These games emphasize simplicity and creativity, fostering a sense of ownership and satisfaction without the pressure of traditional gaming objectives.

The immersive environments created by cozy games are enhanced by their aesthetic appeal and calming audio. Players can design unique spaces that reflect their personal style, contributing to a rewarding gameplay experience. The soothing soundscapes and music options available in these games further enhance their relaxing qualities, making them ideal for unwinding after a long day [1]. 

The growing community surrounding cozy gaming is also noteworthy. Curators and content creators actively promote upcoming titles, helping players discover new experiences that deliver comforting vibes. Games like "Hello Kitty Island Adventure" and "CraftCraft: Fantasy Merchant Simulator" are gaining attention, showcasing the genre's diversity and appeal [2]. This community-driven approach fosters a sense of belonging among players who seek solace in these virtual spaces, reinforcing the idea that cozy games are not just a niche but a staple for many.

Popular titles such as "Stardew Valley" and "Animal Crossing: New Horizons" continue to be recommended for their engaging yet low-pressure gameplay, making them ideal for casual gamers [5]. The extensive hours of gameplay invested in cozy titles suggest that their appeal extends beyond mere relaxation; they also provide opportunities for shared experiences and community interaction [3]. 

In conclusion, cozy simulation games are redefining the gaming landscape by offering players a chance to relax, create, and connect. Their immersive qualities and focus on player engagement resonate with a wide audience, providing a much-needed escape from the stresses of everyday life. As the genre continues to evolve, upcoming titles like "Fields of Mistria," "Pine Hearts," and "Garden Life: A Cozy Simulator" promise to enrich the cozy gaming experience even further [6].


---

## Conclusion

As we navigate through 2024, the cozy gaming genre on Steam continues to thrive, offering players a delightful escape from the pressures of traditional gaming. This report has highlighted the charm of cozy indie games, showcasing titles that prioritize relaxation, creativity, and community engagement. From city-building simulators to immersive productivity games, the variety of experiences available caters to a wide audience seeking comfort and connection. The growing recognition of cozy games as a legitimate genre reflects a broader trend in gaming, where emotional well-being takes precedence. As players embrace these soothing experiences, the cozy gaming community is set to flourish, inviting more individuals to discover the joy of unwinding in beautifully crafted virtual worlds.

## Sources
[1] https://store.steampowered.com/curator/42587990-Just-Some-Cozy-Gaming/?snr=1_1050_curatorsreviewing_  
[2] https://store.steampowered.com/curator/43535555-Camp-Cozy-Games/?snr=1_1050_curatorsreviewing_  
[3] https://www.youtube.com/watch?v=2tla2XjiV0c  
[4] https://www.wired.com/gallery/best-cozy-games/  
[5] https://www.self.com/story/best-cozy-games  
[6] https://www.dualshockers.com/best-cozy-games-of-2024/

Node 'finalize_report' execution ended.
